In [ ]:
!pip install biopython

from Bio import Entrez, SeqIO
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 12.6 MB/s eta 0:00:00


In [ ]:
Entrez.email = "jacquelinekgrimm@gmail.com"

# Get gene sequences from NCBI using accession number
def get_genes(accession):
    handle = Entrez.efetch(db="nucleotide", id=accession, rettype="gb", retmode="text")
    record = SeqIO.read(handle, "genbank")
    genes = {}
    for feature in record.features:
        if feature.type == "CDS":
            gene_name = ''
            # Check for gene names after both 'gene' and 'locus_tag'
            if 'gene' in feature.qualifiers:
                gene_name = feature.qualifiers['gene'][0]
            elif 'locus_tag' in feature.qualifiers:
                gene_name = feature.qualifiers['locus_tag'][0]
            else:
                gene_name = f"Unknown_gene_{len(genes)+1}"
            genes[gene_name] = str(feature.location.extract(record).seq)
    return genes

# Get Salmonella and Bacillus genes
salmonella_genes = get_genes("AL513382")
bacillus_genes = get_genes("AE016877")

# Print sequences of ten genes in each class
salmonella_gene_names = list(salmonella_genes.keys())[:10]
bacillus_gene_names = list(bacillus_genes.keys())[:10]

print("Salmonella Genes:")
for gene_name in salmonella_gene_names:
    print(f"{gene_name}: {salmonella_genes[gene_name]}")

print("\nBacillus Genes:")
for gene_name in bacillus_gene_names:
    print(f"{gene_name}: {bacillus_genes[gene_name]}")

Salmonella Genes:
STY0001: ATGAACCGCATCAGCACCACCACCATTACCACCATCACCATTACCACAGGTAACGGTGCGGGCTGA
STY0002: ATGCGAGTGTTGAAGTTCGGCGGTACATCAGTGGCAAATGCAGAACGTTTTCTGCGTGTTGCCGATATTCTGGAAAGCAATTCCAGGCAAGGGCAGGTAGCGACCGTACTTTCCGCCCCCGCGAAAATTACCAACCATCTGGTGGCGATGATTGAAAAAACTATCGGCGGCCAGGATGCTTTGCCGAATATCAGCGATGCCGAACGTATTTTTTCTGACCTGCTCGCAGGACTTGCCAGCGCGCAGCCGGGATTCCCGCTTGCACGGTTGAAAATGGTTGTCGAACAAGAATTCGCTCAGATCAAACATGTTTTGCATGGTATCAGCCTGCTGGGTCAGTGCCCGGATAGCATCAACGCCGCGCTGATTTGCCGTGGCGAAAAAATGTCGATCGCGATTATGGCGGGACTCCTGGAGGCGCGTGGACATCGCGTCACGGTGATCGATCCGGTAGAAAAACTGCTGGCGGTGGGCCATTACCTTGAATCTACCGTCGATATCGCGGAATCGACTCGCCGTATCGCCGCCAGCCAGATCCCGGCCGATCACATGATCCTGATGGCGGGCTTTACTGCCGGTAATGAAAAGGGTGAACTGGTGGTGCTGGGCCGTAATGGTTCCGACTATTCCGCCGCCGTGCTGGCCGCCTGTTTACGCGCTGACTGCTGTGAAATCTGGACTGACGTCGATGGCGTGTATACCTGTGACCCGCGCCAGGTGCCGGACGCCAGGCTGTTGAAATCGATGTCCTACCAGGAAGCGATGGAGCTCTCTTACTTCGGCGCTAAAGTCCTTCACCCTCGCACCATAACGCCTATCGCCCAGTTCCAGATCCCCTGTCTGATTAAAAATACCGGCAATCCGCAGGCGCCAGGAACGCTGATCGGCGCGTCCAGC

In [ ]:
# Function to create k-mers
def make_kmers(seq, size):
    return [seq[x:x + size].lower() for x in range(len(seq) - size + 1)]

# Function to join k-mer words into sentences
def sentences(genes_dict, kmer_size):
    gene_sentences = {}
    for gene_name, sequence in genes_dict.items():
        words = make_kmers(sequence, size=kmer_size)
        joined_sentence = ' '.join(words)
        gene_sentences[gene_name] = joined_sentence
    return gene_sentences

# Creating sentences
salmonella_sentences = sentences(salmonella_genes, kmer_size=6)
bacillus_sentences = sentences(bacillus_genes, kmer_size=6)

# Print the joined k-mer sentences for the first gene in each class
print("Salmonella Gene Sentence:")
print(salmonella_sentences[salmonella_gene_names[0]])
print("\nBacillus Gene Sentence:")
print(bacillus_sentences[bacillus_gene_names[0]])

Salmonella Gene Sentence:
atgaac tgaacc gaaccg aaccgc accgca ccgcat cgcatc gcatca catcag atcagc tcagca cagcac agcacc gcacca caccac accacc ccacca caccac accacc ccacca caccat accatt ccatta cattac attacc ttacca taccac accacc ccacca caccat accatc ccatca catcac atcacc tcacca caccat accatt ccatta cattac attacc ttacca taccac accaca ccacag cacagg acaggt caggta aggtaa ggtaac gtaacg taacgg aacggt acggtg cggtgc ggtgcg gtgcgg tgcggg gcgggc cgggct gggctg ggctga

Bacillus Gene Sentence:
ttggaa tggaaa ggaaaa gaaaat aaaata aaatat aatatc atatct tatctc atctct tctctg ctctga tctgat ctgatt tgattt gattta atttat tttatg ttatgg tatgga atggaa tggaat ggaata gaatag aatagt atagtg tagtgc agtgcc gtgcct tgcctt gcctta ccttaa cttaaa ttaaaa taaaag aaaaga aaagaa aagaat agaatt gaatta aattag attaga ttagaa tagaaa agaaaa gaaaaa aaaaaa aaaaaa aaaaag aaaagg aaaggt aaggta aggtaa ggtaag gtaagc taagca aagcaa agcaag gcaagc caagcc aagcct agccta gcctag cctagt ctagtt tagtta agttat gttatg ttatga tatgag atgaga tgagac gagaca agacat gaca

In [ ]:
# Creating labels for Salmonella and Bacillus genes
labels_salmonella = ['Salmonella'] * len(salmonella_genes)
labels_bacillus = ['Bacillus'] * len(bacillus_genes)

# Combining gene sentences and labels
all_genes = list(salmonella_sentences.values()) + list(bacillus_sentences.values())
all_labels = labels_salmonella + labels_bacillus

# Splitting the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(all_genes,
                                                    all_labels,
                                                    test_size=0.2,
                                                    random_state=42)

In [ ]:
# Generating bag of words models
cv = CountVectorizer(ngram_range=(4, 4))
X_train_bow = cv.fit_transform(X_train)
X_test_bow = cv.transform(X_test)

# Training the classifier
classifier = MultinomialNB(alpha=0.1)
classifier.fit(X_train_bow, y_train)

# Predicting classes of the test set
predictions = classifier.predict(X_test_bow)

# Evaluating the classifier
accuracy = classifier.score(X_test_bow, y_test)
print(f"Accuracy: {accuracy}")

Accuracy: 0.973953013278856


In [ ]:
# Testing a Salmonella sequence from a different strain (Accession: FQ312003)
sequence = "ATGACAGAGTACACAACATCCATGAACCGCATCAGCACCACCACCATTACCACCATCACCATTACCACAGGTAACGGTGCGGGCTGA"

# Converting the sequence into k-mers
kmer_size = 6
sequence_kmers = make_kmers(sequence, kmer_size)

# Joining k-mers into a sentence
sequence_sentence = ' '.join(sequence_kmers)

# Creating a bag of words
sequence_bow = cv.transform([sequence_sentence])

# Predicting the class of the sequence
predicted_class = classifier.predict(sequence_bow)
print("Predicted class:", predicted_class[0])

Predicted class: Salmonella


In [ ]:
# Testing a Bacillus sequence from a different strain (Accession: CP010053)
sequence = "ATGAAATTTACGATTCAAAAAGATCGTTTAGTCGAAAGTGTACAAGACGTTCTAAAAGCTGTTTCATCCAGAACAACAATTCCCATTC"

# Converting the sequence into k-mers
kmer_size = 6
sequence_kmers = make_kmers(sequence, kmer_size)

# Joining k-mers into a sentence
sequence_sentence = ' '.join(sequence_kmers)

# Creating a bag of words
sequence_bow = cv.transform([sequence_sentence])

# Predicting the class of the sequence
predicted_class = classifier.predict(sequence_bow)
print("Predicted class:", predicted_class[0])

Predicted class: Bacillus
